<a href="https://colab.research.google.com/github/KV-23/FPR/blob/main/stock_prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program will predict stock prices using ML

In [ ]:
!pip install quandl
import quandl
quandl.ApiConfig.api_key = "5skFHDPceDFrgifZwy5g"

In [ ]:
#install dependancies
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [ ]:
#Get the stock data
df=quandl.get("WIKI/FB")
#take a look at the data
print(df.head())

             Open   High    Low    Close       Volume  Ex-Dividend  \
Date                                                                 
2012-05-18  42.05  45.00  38.00  38.2318  573576400.0          0.0   
2012-05-21  36.53  36.66  33.00  34.0300  168192700.0          0.0   
2012-05-22  32.61  33.59  30.94  31.0000  101786600.0          0.0   
2012-05-23  31.37  32.50  31.36  32.0000   73600000.0          0.0   
2012-05-24  32.95  33.21  31.77  33.0300   50237200.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2012-05-18          1.0      42.05      45.00     38.00     38.2318   
2012-05-21          1.0      36.53      36.66     33.00     34.0300   
2012-05-22          1.0      32.61      33.59     30.94     31.0000   
2012-05-23          1.0      31.37      32.50     31.36     32.0000   
2012-05-24          1.0      32.95      33.21     31.77     33.0300   

           

In [ ]:
#Get the adjusted close price
df= df[['Adj. Close']]
#see the data using df.head
print(df.head())

            Adj. Close
Date                  
2012-05-18     38.2318
2012-05-21     34.0300
2012-05-22     31.0000
2012-05-23     32.0000
2012-05-24     33.0300


In [ ]:
#number of days out into the future we want to predict
forecast_out =30
#create another column to shif the variable upwards.
df['Prediction']=df[['Adj. Close']].shift(-forecast_out)
#print the shifted data
print(df.tail())

            Adj. Close  Prediction
Date                              
2018-03-21      169.39         NaN
2018-03-22      164.89         NaN
2018-03-23      159.39         NaN
2018-03-26      160.06         NaN
2018-03-27      152.19         NaN


In [ ]:
#Create the Independant dataset x
##convert tha dataframe into a numpy array
X= np.array(df.drop(['Prediction'],axis=1))
#remove the last n rows
X= X[:-forecast_out]
print(X)

In [ ]:
###Create the variable y dataset
##convert the dataframe to a numpy array (All values including NaNs)

y=np.array(df['Prediction'])
## get all the y values except last n rows
y=y[:-forecast_out]
print(y)

[ 30.771  31.2    31.47  ... 159.39  160.06  152.19 ]


In [ ]:
#split data into 80%training and 20% testing
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
svr_rbf=SVR(kernel="rbf",C=1e3,gamma=0.1)
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, gamma=0.1)

In [ ]:
#Testing the model : score returns the confidence of deermination of R^2 of prediction
#The best score is 1
svr_confidence=svr_rbf.score(x_test,y_test)
print("Svr_confidence score: ",svr_confidence)

Svr_confidence score:  0.9844232421473244


In [ ]:
#comparing with linear regression
lr=LinearRegression()
#Train the model
lr.fit(x_train,y_train)

In [ ]:
#Testing the model : score returns the confidence of deermination of R^2 of prediction
#The best score is 1
lr_confidence=lr.score(x_test,y_test)
print("Lr_confidence score: ",lr_confidence)

Lr_confidence score:  0.9797574226366333


In [ ]:
#set x_forecast equal to the last 30 rows of the original data (excluding the 'Prediction' column)
x_forecast=np.array(df.drop(['Prediction'], axis=1))[-forecast_out:]
print(x_forecast)

In [ ]:
#lr predictions for the next "n" days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[177.12925852 183.56815674 184.0129158  181.38479408 180.02019242
 181.94074291 183.03242424 187.37893324 189.03667156 185.52913987
 182.35517749 179.94943529 180.63679021 184.45767486 183.83096892
 187.80347598 186.418658   189.33991637 188.86483283 185.95368261
 188.28866768 187.95509839 189.19840213 176.53287705 172.07517829
 173.32859019 168.77991797 163.22042971 163.89767647 155.94255417]


In [ ]:
#svr_predictions for next "n" days
svr_predictions = svr_rbf.predict(x_forecast)
print(svr_predictions)

[178.32691904 181.46732677 181.67713482 174.90804657 173.05021283
 177.13026569 180.84858244 185.30186127 179.11318741 185.32581855
 178.84994791 173.08966553 173.18239113 182.10548584 181.59170102
 183.05439288 187.64989879 179.39791805 179.17873637 186.80953251
 180.66414025 182.24435363 179.20573262 179.01814577 172.63009165
 172.82233966 172.18964019 167.84350812 166.19386024 157.73010414]
